# Lab 2
We are implementing image augmentation on the SimpleNN model.


In [2]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# define the SimpleNN mode;
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1   = nn.Linear(16*6*6, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1) # flatten
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

## Sanity check
Here is a sanity check to verify the implementation of SimpleNN. 

In [6]:
# sanity check for the correctness of SimpleNN

# create 10 sets of dummy inputs with same size as CIFAR-10 images (they are 32x32 RGB images with 3 channels)
import random
rand_num_images = random.sample(range(1, 100), 10)

for i in rand_num_images:
    # print(f"\nTESTING WITH {i} IMAGES")
    dummy_input = torch.rand(i, 3, 32, 32)

    # test each dummy set to make sure the final output is of correct size (expecting number of images x 10)
    test_model = SimpleNN()
    outputs = test_model(dummy_input)
    # print(f"Output size: {outputs.size()}, expected: [{i} x 10]")
    assert outputs.size() == (i, 10), "Not sane, pls fix"
    
    # test number of parameters of all CONV and FC layers of the model
    # This should not change with different number of images
    expected_total_num_params = 0
    actual_total_num_params = 0
    for name, module in test_model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            # Get the weight of the module as a NumPy array
            weight = module.weight.detach().cpu().numpy()
            # calculate the number of parameters
            expected_num_params = 1
            for w in weight.shape:
                expected_num_params *= w
            expected_total_num_params += expected_num_params
            actual_total_num_params += weight.size
            # print(f"{name} num_Params: {weight.size}, expected: {expected_num_params}")
            assert weight.size == expected_num_params, "Not sane, pls fix"
            # This does not count bias
    # print(f"Total num_Params: {actual_total_num_params}, expected: {expected_total_num_params}")

print("All tests passed!")

All tests passed!


## Question 1: Random Crop + Random Flipping
Train a model with 1) random cropping with a padding of 4 and 2) random ﬂipping

 Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [8]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
transform_train = transforms.Compose([
        transforms.ToTensor(), # convert PIL to Tensor
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=32, padding=4),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # normalize the image
    ])
transform_val = transforms.Compose([
        transforms.ToTensor(), # convert PIL to Tensor
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # normalize the image
    ])
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [5]:
# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    mode='train', 
    download=True,
    transform=transform_train,
)

val_set = CIFAR10(
    root=DATA_ROOT, 
    mode='val', 
    download=True,
    transform=transform_val,
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=TRAIN_BATCH_SIZE, 
    shuffle=True,
    num_workers=4
)
val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE, 
    shuffle=False, # Does not matter here
    num_workers=4
)
############################################

Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [6]:
# specify the device for computation
#############################################
# your code here
if torch.cuda.is_available(): # check if NVIDIA GPU is available
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available(): # check if Apple's Metal is available
    device = torch.device("mps")
    print(f"Checking pytorch is built with mps activated: {torch.backends.mps.is_built()}")
else:
    device = torch.device("cpu")

if device == torch.device("cuda:0") or device == torch.device("mps"):
    print(f"Running on {device} GPU...")
else:
    print("Running on CPU...")

# instantiate the model on device
model = SimpleNN().to(device)

#############################################

Checking pytorch is built with mps activated: True
Running on mps GPU...


## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [7]:
import torch.nn as nn
import torch.optim as optim

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.01

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(params=model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


batch is better when you have larger lrs

In [8]:
# some hyperparameters
# total number of training epochs
from http.cookiejar import DefaultCookiePolicy
import pickle # for saving accuracy and loss

EPOCHS = 30

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0
current_learning_rate = INITIAL_LR

# store train, val accuracy per epoch for plotting
avg_train_acc = []
avg_train_loss = []
avg_val_acc = []
avg_val_loss = []

# BONUS: hyperparams for learning rate scheduler
DECAY_EPOCHS = 10 
DECAY = 0.1

print("==> Training starts!")
print("="*50)


for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    # BONUS
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
    
    #######################
    # your code here
    # switch to train mode
    model.train()
    
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want

    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # your code here
        # copy inputs to device
        inputs = inputs.to(device)
        
        # compute the output and loss
        outputs = model(inputs)
        targets = targets.to(device) # don't forget to also copy targets to device
        loss = criterion(outputs, targets)
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        loss.backward()
        
        # apply gradient and update the weights
        optimizer.step()
        
        # count the number of correctly predicted samples in the current batch
        _, preds = outputs.max(1)
        total_examples += targets.shape[0]
        correct_examples += preds.eq(targets).sum().item()
        # track loss
        train_loss += loss.item()

        ####################################
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # store train accuracy and loss
    avg_train_loss.append(avg_loss)
    avg_train_acc.append(avg_acc)

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    model.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs = inputs.to(device)
            
            # compute the output and loss
            outputs = model(inputs)
            targets = targets.to(device) # don't forget to also copy targets to device
            loss = criterion(outputs, targets)
            
            # count the number of correctly preds samples in the current batch
            _, preds = outputs.max(1)
            total_examples += targets.shape[0]
            correct_examples += preds.eq(targets).sum().item()
            # track loss
            val_loss += loss.item()

            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # store val accuracy and loss
    avg_val_loss.append(avg_loss)
    avg_val_acc.append(avg_acc)

    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
           os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': model.state_dict(),
                'epoch': i,
                'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0:
Training loss: 1.8529, Training accuracy: 0.3105
Validation loss: 1.5663, Validation accuracy: 0.4320
Saving ...
Epoch 1:


# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```

In [ ]:
# save/load the training and validation accuracy and loss
SAVE_NEW_DATA = False
if SAVE_NEW_DATA:
    with open(os.path.join(CHECKPOINT_FOLDER, 'train_acc.pkl'), 'wb') as f:
        pickle.dump(avg_train_acc, f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'train_loss.pkl'), 'wb') as f:
        pickle.dump(avg_train_loss, f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'val_acc.pkl'), 'wb') as f:
        pickle.dump(avg_val_acc, f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'val_loss.pkl'), 'wb') as f:
        pickle.dump(avg_val_loss, f)
else:
    with open(os.path.join(CHECKPOINT_FOLDER, 'train_acc.pkl'), 'rb') as f:
        avg_train_acc = pickle.load(f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'train_loss.pkl'), 'rb') as f:
        avg_train_loss = pickle.load(f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'val_acc.pkl'), 'rb') as f:
        avg_val_acc = pickle.load(f)
    with open(os.path.join(CHECKPOINT_FOLDER, 'val_loss.pkl'), 'rb') as f:
        avg_val_loss = pickle.load(f)


In [ ]:
# plot the training and validation loss
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.lineplot(x=range(len(avg_train_loss)), y=avg_train_loss, label='train loss', ax=ax)
sns.lineplot(x=range(len(avg_val_loss)), y=avg_val_loss, label='val loss', ax=ax)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training and validation loss')
sns.despine()